In [1]:
import pandas as pd
from wmfdata import hive
from wmfdata.utils import pct_str

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


## Global South new editor retention

In [2]:
hive.run("show partitions wmf.geoeditors_daily")

,partition
0,month=2018-11
1,month=2018-12
2,month=2019-01
3,month=2019-02
4,month=2019-03
5,month=2019-04


In [5]:
# The new_editor table must be updated for the cohort month (using 01b-new_editor_table.ipynb)
# before this can be successfully run 
gs_new_editors = hive.run("""
select 
    user_name, 
    wiki,
    1st_month_edits,
    2nd_month_edits
from neilpquinn.new_editors ne
left join wmf.geoeditors_daily gd
on
    ne.user_id = gd.user_fingerprint_or_id and
    ne.wiki = gd.wiki_db and
    ne.cohort = gd.month
left join canonical_data.countries cdc
on gd.country_code = cdc.iso_code
where
    ne.cohort = "{retention_cohort}" and
    gd.month = "{retention_cohort}" and
    economic_region = "Global South"
group by user_name, wiki, 1st_month_edits, 2nd_month_edits
""".format(
    retention_cohort="2019-02"
))

In [4]:
gs_second_mo_eds = gs_new_editors[lambda df: df["2nd_month_edits"] >= 1]
print(pct_str(len(gs_second_mo_eds) / len(gs_new_editors), decimals=2))

ZeroDivisionError: division by zero